In [5]:
# Import required modules
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras import backend as K
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import *
from keras.optimizers import RMSprop
from keras.regularizers import l2
from generator import Generator
# from models import *
import h5py
import numpy as np
import sys
import os

# Set Tensorflow backend to avoid full GPU pre-loading
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# set_session(tf.Session(config = config))

In [6]:
print "Loading Generator"
# Hardcoding because codes will mostly run from my account.
# Also adding sys.argv stuff with defaults is a pain.
gen = Generator(dataset_directory = '/home/siddhartha.l/Show-and-Tell/data')

Loading Generator


In [3]:
print "Generating model (This will take while)"
model = get_model_no_atenttion(
            gen.img_feature_size, 
            gen.vocab_size, 
            gen.max_token_len, 
            embedding_dim = gen.embedding_size)
model.compile('adamax', loss='categorical_crossentropy', metrics=['accuracy'])

Generating model (This will take while)


NameError: name 'gen' is not defined

In [4]:
batch_size = 64
print "Start training"
# Save model for every 10 epochs
prev_loss = 10
for i in range(1,10):
    hist = model.fit_generator(
                gen.pullData(),
                epochs=10,
                steps_per_epoch=int(gen.training_samples_count / batch_size), 
                shuffle = True, verbose = 1#, callbacks = [checkpointer]
                )
    if hist.history['loss'][-1] < prev_loss:
        prev_loss = hist.history['loss'][-1]
        print 'Saving ~models/attention_flickr_epoch_' + str(i*10) + '.h5'
        model.save('/home/siddhartha.l/Show-and-Tell/data/models/attention_flickr_epoch_' + str(i*10) + '.h5')

Start training
Epoch 1/10
632/632 [==============================] - 316s - loss: 1.1617 - acc: 0.7968   
Epoch 2/10
632/632 [==============================] - 315s - loss: 0.8737 - acc: 0.8215   
Epoch 3/10
632/632 [==============================] - 315s - loss: 0.7721 - acc: 0.8322   
Epoch 4/10
632/632 [==============================] - 311s - loss: 0.6852 - acc: 0.8427   
Epoch 5/10
632/632 [==============================] - 310s - loss: 0.6241 - acc: 0.8508   
Epoch 6/10
632/632 [==============================] - 309s - loss: 0.5635 - acc: 0.8603   
Epoch 7/10
632/632 [==============================] - 314s - loss: 0.5073 - acc: 0.8696   
Epoch 8/10
632/632 [==============================] - 312s - loss: 0.4700 - acc: 0.8764   
Epoch 9/10
632/632 [==============================] - 315s - loss: 0.4334 - acc: 0.8838   
Epoch 10/10
632/632 [==============================] - 313s - loss: 0.3958 - acc: 0.8917   
Saving ~models/attention_flickr_epoch_10.h5
Epoch 1/10
632/632 [==========

RuntimeError: Can't decrement id ref count (unable to close file, errno = 122, error message = 'Disk quota exceeded')

In [ ]:
model.save('../data/att_tmp.h5')

In [18]:
# Load previously saved model if necessary
from keras.models import load_model

model = load_model('../data/flicker8k/flickr_show_and_tell_epoch_10.h5')

In [ ]:
from IPython.display import Image
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input

In [ ]:
### For runtime testing only ###

# from process_images import *

base_model = InceptionV3(weights = 'imagenet', include_top = True, input_shape = (299, 299, 3))
img_model =  Model(
        input = base_model.input,
        outputs = [base_model.get_layer('mixed10').output])
target_size = (299, 299)
print img_model.output_shape
output_shape = img_model.output_shape[1:]

In [ ]:
model.summary()

In [22]:
image_filenames = ['2021613437_d99731f986.jpg']
dataset_directory = '../data/flicker8k'

from keras.backend.tensorflow_backend import get_session

preprocessed_images = []
# Remove repeating filenames (if any)
# image_filenames = list(set(image_filenames))
number_of_images = len(image_filenames)
img_input = []

# Iterate over all images and preprocess them
for img_id, img_name in enumerate(image_filenames): # For coco make 3D array , do batch
    img_filepath = dataset_directory + '/Flickr8k_Dataset/' + img_name
    # Image preprocessing
    img = image.load_img(img_filepath, target_size = target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    preprocessed_images.append(np.squeeze(img))

preprocessed_images = np.asarray(preprocessed_images)
img_features = img_model.predict(preprocessed_images)
# img_features = np.asarray([img_features])

text_in = np.zeros((1,gen.max_token_len))
text_in[0][0] = gen.token_to_id['<start>']

predictions = []
activations = []
for arg in range(gen.max_token_len-1):
    pred = model.predict([img_features, text_in])
    tok = np.argmax(pred[0][arg])
    word = gen.id_to_token[tok]
    text_in[0][arg+1] = tok
    if word == '<end>':
        break
    predictions.append(word)

print ' '.join(predictions) + '.'
# print att_map
Image(filename= dataset_directory + '/Flickr8k_Dataset/' + image_filenames[0])

NameError: name 'image' is not defined

In [ ]:
image_filenames = ['2021613437_d99731f986.jpg']
dataset_directory = '../data/flicker8k'

from matplotlib.pyplot import imshow
from matplotlib.pyplot import imread
from matplotlib import pyplot
% matplotlib inline

preprocessed_images = []
# Remove repeating filenames (if any)
# image_filenames = list(set(image_filenames))
number_of_images = len(image_filenames)
img_input = []

# Iterate over all images and preprocess them
for img_id, img_name in enumerate(image_filenames): # For coco make 3D array , do batch
    img_filepath = dataset_directory + '/Flickr8k_Dataset/' + img_name
    # Image preprocessing
    img = image.load_img(img_filepath, target_size = target_size)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    preprocessed_images.append(np.squeeze(img))

preprocessed_images = np.asarray(preprocessed_images)
img_features = img_model.predict(preprocessed_images)
# img_features = np.asarray([img_features])

text_in = np.zeros((1,gen.max_token_len))
text_in[0][0] = gen.token_to_id['<start>']
text_in[0][1] = gen.token_to_id['a']
text_in[0][2] = gen.token_to_id['little']
# text_in[0][3] = gen.token_to_id['and']
# text_in[0][4] = gen.token_to_id['a']

layer_name = 'time_distributed_6'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
# intermediate_output = intermediate_layer_model.predict(data)

predictions = []
activations = []
# for arg in range(gen.max_token_len-1):
pred = intermediate_layer_model.predict([img_features, text_in])
#     tok = np.argmax(pred[0][arg])
#     word = gen.id_to_token[tok]
#     text_in[0][arg+1] = tok
#     if word == '<end>':
#         break
#     predictions.append(word)

# print ' '.join(predictions) + '.'
# print att_map
# Image(filename= dataset_directory + '/Flickr8k_Dataset/' + image_filenames[0])
# print pred.shape
# pyplot.subplot()
att = pred[0,3,:].reshape((8,8), order = 'A')#, np.ones((immmg.shape[0], immmg.shape[1])))
# att.shape
immmg = imread(dataset_directory + '/Flickr8k_Dataset/' + image_filenames[0])
imshow(immmg)
imshow(att, alpha = 0.5, interpolation='bilinear', extent=[0, immmg.shape[1], immmg.shape[0], 0])
# imshow(pred[0,2,:].reshape(8,8))
# imshow(pred[0,1,:].reshape(8,8))

In [ ]:
imshow(pred[0,0,:].reshape(8,8))

In [ ]:
imshow(pred[0,0,:].reshape(8,8))

In [20]:
# For generating test results in bulk

from IPython.display import Image
import time

with open('../data/flicker8k/preprocessed/test_captions.txt') as captions_file:
    captions = captions_file.read().split('\n')
    
class Caption:
    def __init__(self, name):
        self.name = name
        self.captions = ['','','','','']
        self.result = ''
        
    def add(self, caption_number, caption):
        self.captions[caption_number] = caption
        
test_results = {}

for caption in captions:
    if len(caption) < 5:
        continue
    caption = caption.split('\t')
    img_name = caption[0].split('#')
    caption_number = int(img_name[1])
    img_name = img_name[0]
    caption = caption[1].lower()
    
    try:
        cap_obj = test_results[img_name]
        cap_obj.add(caption_number, caption)
    except Exception as e:
#         print str(e)
        feature_dataset = h5py.File('/home/siddhartha.l/Show-and-Tell/data/flicker8k/preprocessed/flickr_8k_test_features.hf', 'r')
        img_features = feature_dataset[img_name]['cnn_features'][:]

        # image_filenames = get_image_filenames(dataset_directory + '/' + img_list_file)
        # print img_features.shape
        features = np.array([img_features])

        text_in = np.zeros((1,gen.max_token_len))
        text_in[0][:] = np.full((gen.max_token_len,), 0)
        text_in[0][0] = 4230

        # print features,text_in
        arr = []
        zeros = np.array([np.zeros(512)])
        for arg in range(gen.max_token_len-1):
            pred = model.predict([features, zeros, text_in])
            tok = np.argmax(pred[0][arg])
            word = gen.id_to_token[tok]
            text_in[0][arg+1] = tok
            if word == '<end>':
                break
            arr.append(word)
        arr.append('.')
        cap_obj = Caption(img_name)
        cap_obj.add(caption_number, caption)
        cap_obj.result = ' '.join(arr)
        test_results.update({img_name: cap_obj})
        import pickle
        pickle.dump(test_results, open('../data/flicker8k/preprocessed/test_results_140.p', 'wb') )

In [19]:

from IPython.display import Image
import time

with open('../data/flicker8k/preprocessed/test_captions.txt') as captions_file:
    captions = captions_file.read().split('\n')
    
class Caption:
    def __init__(self, name):
        self.name = name
        self.captions = ['','','','','']
        self.result = ''
        
    def add(self, caption_number, caption):
        self.captions[caption_number] = caption
        
test_results = {}
count = 0 
beam_size = 3

for caption in captions:
    count+=1
    if(count > 10):
        break
    if len(caption) < 5:
        continue
    caption = caption.split('\t')
    img_name = caption[0].split('#')
    caption_number = int(img_name[1])
    img_name = img_name[0]
    caption = caption[1].lower()
    
    try:
        cap_obj = test_results[img_name]
        cap_obj.add(caption_number, caption)
    except Exception as e:
        feature_dataset = h5py.File('/home/siddhartha.l/Show-and-Tell/data/flicker8k/preprocessed/flickr_8k_test_features.hf', 'r')
        img_features = feature_dataset[img_name]['cnn_features'][:]
        features = np.array([img_features])

        text_in = np.zeros((beam_size,1,gen.max_token_len))
        for i in range(beam_size):
            text_in[i][0][:] = np.full((gen.max_token_len,), 0)
            text_in[i][0][0] = 4230
        
        temp_array = np.zeros((beam_size*beam_size,1,gen.max_token_len))
        
        for i in range(beam_size*beam_size):
            temp_array[i][0][:] = np.full((gen.max_token_len,), 0)
            temp_array[i][0][0] = 4230
 
        temp_beam = np.ones(beam_size*beam_size)
    
        beam1_prob = 1.0
        beam2_prob = 1.0
        beam3_prob = 1.0 
        
        pred = model.predict([features, zeros, text_in[0]])
        tok = np.argsort(pred[0][arg])[1:][::-1]
        text_in[0][0][1] = tok[0]
        text_in[1][0][1] = tok[1]
        text_in[2][0][1] = tok[2]
        beam1_prob *= pred[0][arg][tok[0]]
        beam2_prob *= pred[0][arg][tok[1]]
        beam3_prob *= pred[0][arg][tok[2]] 

        # print features,text_in
        arr = []
        zeros = np.array([np.zeros(512)])
        for arg in range(1,gen.max_token_len-1):                
            for i in range(beam_size):
                pred = model.predict([features, zeros, text_in[i]])
                tok = np.argsort(pred[0][arg])[1:][::-1]
                temp_beam[i*beam_size + 0] *= pred[0][arg][tok[0]]
                temp_beam[i*beam_size + 1] *= pred[0][arg][tok[1]]
                temp_beam[i*beam_size + 2] *= pred[0][arg][tok[2]]
                temp_array[i*beam_size + 0][0][arg+1] = tok[0]
                temp_array[i*beam_size + 1][0][arg+1] = tok[1]
                temp_array[i*beam_size + 2][0][arg+1] = tok[2]            
            
                print (temp_beam)
                word = gen.id_to_token[tok[0]]
                text_in[i][0][arg+1] = tok[0]                
                if word == '<end>':
                    break
                arr.append(word)
            beam = np.argsort(temp_beam)[::-1]
            print (beam)
            print arr
        arr.append('.')
   


        cap_obj = Caption(img_name)
        cap_obj.add(caption_number, caption)
        cap_obj.result = ' '.join(arr)
        test_results.update({img_name: cap_obj})


[ 0.38203436  0.19313422  0.12713626  1.          1.          1.          1.
  1.          1.        ]
[ 0.38203436  0.19313422  0.12713626  0.27878559  0.20312086  0.16682807
  1.          1.          1.        ]
[ 0.38203436  0.19313422  0.12713626  0.27878559  0.20312086  0.16682807
  0.79327369  0.05798008  0.04636115]
[6 0 3 4 1 5 2 7 8]
['boy', 'little', 'boy']
[ 0.1324315   0.03217717  0.01434836  0.27878559  0.20312086  0.16682807
  0.79327369  0.05798008  0.04636115]
[ 0.1324315   0.03217717  0.01434836  0.24749379  0.0087169   0.0070565
  0.79327369  0.05798008  0.04636115]
[ 0.1324315   0.03217717  0.01434836  0.24749379  0.0087169   0.0070565
  0.2090093   0.0118527   0.00893249]
[3 6 0 1 2 7 8 4 5]
['boy', 'little', 'boy', 'in', 'boy', 'in']
[  1.23575923e-01   6.05165769e-04   2.27963660e-04   2.47493790e-01
   8.71690467e-03   7.05649918e-03   2.09009299e-01   1.18526956e-02
   8.93249241e-03]
[ 0.12357592  0.00060517  0.00022796  0.17395562  0.00098957  0.00045035
  0.2

[  4.99657434e-09   2.04949441e-15   2.34885096e-20   8.35609071e-05
   4.70303681e-13   2.09550798e-15   2.84037282e-05   5.37167963e-13
   1.50609437e-14]
[3 6 0 7 4 8 5 1 2]
['boy', 'little', 'boy', 'in', 'boy', 'in', 'a', 'is', 'a', 'blue', 'wearing', 'blue', 'shirt', 'a', 'shirt', 'is', 'black', 'and', 'smiling', 'tshirt', 'blue', 'whilst', 'and', 'jeans', 'riding', 'holding', 'is', 'a', 'a', 'smiling', 'red', 'green', 'toy', 'toy', 'at']
[  8.46870996e-10   2.21874093e-16   1.47776062e-21   8.35609071e-05
   4.70303681e-13   2.09550798e-15   2.84037282e-05   5.37167963e-13
   1.50609437e-14]
[3 6 0 7 4 8 5 1 2]
['boy', 'little', 'boy', 'in', 'boy', 'in', 'a', 'is', 'a', 'blue', 'wearing', 'blue', 'shirt', 'a', 'shirt', 'is', 'black', 'and', 'smiling', 'tshirt', 'blue', 'whilst', 'and', 'jeans', 'riding', 'holding', 'is', 'a', 'a', 'smiling', 'red', 'green', 'toy', 'toy', 'at']
[  1.39212427e-10   2.59113455e-17   9.30761514e-23   8.35609071e-05
   4.70303681e-13   2.09550798e-15 

[  3.38520074e-21   4.08964356e-31   8.51466640e-41   3.56577337e-15
   8.36795706e-24   1.90813167e-26   2.71287136e-16   2.50615053e-24
   5.38217946e-26]
[3 6 0 4 7 8 5 1 2]
['boy', 'little', 'boy', 'in', 'boy', 'in', 'a', 'is', 'a', 'blue', 'wearing', 'blue', 'shirt', 'a', 'shirt', 'is', 'black', 'and', 'smiling', 'tshirt', 'blue', 'whilst', 'and', 'jeans', 'riding', 'holding', 'is', 'a', 'a', 'smiling', 'red', 'green', 'toy', 'toy', 'at', 'at', 'granite', 'shoulderhigh', 'at', 'granite', 'looming', 'at', 'granite', 'tagged', 'at', 'like', 'receiver']
[  4.62630192e-22   3.88562661e-32   6.60041760e-42   3.56577337e-15
   8.36795706e-24   1.90813167e-26   2.71287136e-16   2.50615053e-24
   5.38217946e-26]
[  4.62630192e-22   3.88562661e-32   6.60041760e-42   1.16801325e-16
   2.37090005e-25   3.86365602e-28   2.71287136e-16   2.50615053e-24
   5.38217946e-26]
[  4.62630192e-22   3.88562661e-32   6.60041760e-42   1.16801325e-16
   2.37090005e-25   3.86365602e-28   6.78007868e-19   4

[  1.07459328e-05   2.33802313e-13   1.08494214e-16   3.68513403e-05
   5.02852547e-09   3.00510487e-11   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[  1.07459328e-05   2.33802313e-13   1.08494214e-16   1.81264263e-05
   1.74906350e-10   9.26409393e-13   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[6 3 0 7 8 4 5 1 2]
['boy', 'young', 'boy', 'in', 'boy', 'in', 'blue', 'in', 'water', 'swim', 'water', 'trunks', 'running', 'for', 'in', 'water', 'over', 'for', 'over']
[  4.67124173e-06   1.32864428e-14   4.59203874e-18   1.81264263e-05
   1.74906350e-10   9.26409393e-13   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[6 3 0 7 8 4 5 1 2]
['boy', 'young', 'boy', 'in', 'boy', 'in', 'blue', 'in', 'water', 'swim', 'water', 'trunks', 'running', 'for', 'in', 'water', 'over', 'for', 'over']
[  1.01914584e-06   1.52471798e-15   4.03415139e-19   1.81264263e-05
   1.74906350e-10   9.26409393e-13   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[  1.01914584e-06   1.52471798e-15

[  1.45350042e-15   1.37760940e-33   8.32315609e-40   2.24385477e-08
   1.14919431e-23   3.27562341e-26   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[  1.45350042e-15   1.37760940e-33   8.32315609e-40   1.06131620e-08
   4.21097841e-25   1.07024460e-27   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[6 7 8 3 0 4 5 1 2]
['boy', 'young', 'boy', 'in', 'boy', 'in', 'blue', 'in', 'water', 'swim', 'water', 'trunks', 'running', 'for', 'in', 'water', 'over', 'for', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over']
[  5.83633463e-16   9.17254491e-35   3.66494046e-41   1.06131620e-08
   4.21097841e-25   1.07024460e-27   3.29573636e-03   3.25662803e-06
   2.35212246e-08]
[6 7 8 3 0 4 5 1 2]
['boy', 'young', 'boy', 'in', 'boy', 'in', 'blue', 'in', 'water', 'swim', 'water', 'trunks', 'running', 'for', 'in', 'water', 'over', 'for', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over', 'over']
[  1.49693586e-16   1.29992128e

In [21]:
# Sample code for importing tested results
import pickle
import nltk

class Caption:
    def __init__(self, name):
        self.name = name
        self.captions = ['','','','','']
        self.result = ''
        
    def add(self, caption_number, caption):
        self.captions[caption_number] = caption
        
test_results = pickle.load(open('../data/flicker8k/preprocessed/test_results_150.p', 'rb'))

# test_results = pickle.load(open('/scratch/siddhartha/COCO/preprocessed/test_results.p', 'rb'))

bleu_score = 0.0
test_set = []
reference_set = []

for img_name in test_results:
    ground_truth = test_results[img_name].captions
    result = test_results[img_name].result
    references = []
    hypothesis = result.split()
    for i in ground_truth:
        references.append(i.split())
    test_set.append(hypothesis)
    reference_set.append(references)

bleu_score = nltk.translate.bleu_score.corpus_bleu(reference_set,test_set)
print ("BLEU4: "+str(bleu_score))
bleu_score = nltk.translate.bleu_score.corpus_bleu(reference_set,test_set,weights = (0.33,0.33,0.33))
print ("BLEU3: "+str(bleu_score))
bleu_score = nltk.translate.bleu_score.corpus_bleu(reference_set,test_set,weights = (0.5,0.5))
print ("BLEU2: "+str(bleu_score))
bleu_score = nltk.translate.bleu_score.corpus_bleu(reference_set,test_set,weights = (1.0,))
print ("BLEU1: "+str(bleu_score))



BLEU4: 0.126406249233
BLEU3: 0.207394425521
BLEU2: 0.332256876917
BLEU1: 0.543800770375
